# Compile thalamic nuclei marker genes from literature sources

In [1]:
import os
import pandas as pd
import anndata as ad

In [2]:
root_dir = '/root/capsule'

## Wang *et al.* 2020 CCFv3 marker genes

In [3]:
ccf_csv = pd.read_csv(os.path.join(root_dir,'resources/Wang_2020_CCFv3_Table_S3_data_supporting_delineation_TH_ZI_only.csv'))

In [4]:
ccf_df = ccf_csv[ccf_csv['Dataset type'].isin(['Tg lines', 'ISH (ABA)'])]

In [5]:
# we don't need to keep all the columns around
col_to_keep = ['structure abbreviation', 
               'full structure name', 
               'Major Brain Division', 
               'Mouse line', 
               'Dataset type', 
               'Transgenic Signal in CCF structure', # this is not actually that accurate for determining marker 
                                                     # genes (what is +/++/+++ relative to??) but I'm keeping it 
                                                     # around just in case it proves useful for something
               'ABA gene name']

ccf_df = ccf_df[col_to_keep]

In [6]:
ccf_df

In [7]:
ccf_df.insert(3, 'Reference', 'Wang et al. 2020')

In [8]:
# clean up the 'mouse line' to just the driver gene for the 'Tg lines' datasets
mask_tg_lines = ccf_df['Dataset type']=='Tg lines'
ccf_df.loc[mask_tg_lines, ['gene symbol']] = ccf_df.loc[mask_tg_lines, ['Mouse line']]['Mouse line'].str.split('-').str[0]

In [9]:
# move 'ABA gene name' over into the 'gene symbol' column to combine with the Tg lines genes
mask_aba = ccf_df['Dataset type']=='ISH (ABA)'
ccf_df.loc[mask_aba, ['gene symbol']] = ccf_df.loc[mask_aba, ['ABA gene name']]['ABA gene name']

In [10]:
# drop the original columns in favor of just having the one "gene symbol" column
ccf_df.drop(labels=['Mouse line','ABA gene name'], axis=1, inplace=True)

In [11]:
# test subset from AD
ccf_df[ccf_df['structure abbreviation']=='AD']

### Cross-reference with brain3 & brain1 gene panels

In [12]:
# get gene panels
brain1_genes_csv = pd.read_csv(os.path.join(root_dir,'resources/gene_panel_VZ147_mouse_609882_brain1.csv'))
brain3_genes_csv = pd.read_csv(os.path.join(root_dir,'resources/gene_panel_VZ142_mouse_638850_brain3.csv'))

genes_brain1 = brain1_genes_csv['gene_panel_VZ147_mouse_609882_brain1'].tolist()
genes_brain3 = brain3_genes_csv['gene_panel_VZ142_mouse_638850_brain3'].tolist()

In [13]:
# flag rows where the gene is in the gene panel
ccf_df['is_in_brain1_gene_panel'] = ccf_df['gene symbol'].isin(genes_brain1)
ccf_df['is_in_brain3_gene_panel'] = ccf_df['gene symbol'].isin(genes_brain3)
ccf_df.head(5)

In [14]:
# example of how to use the gene panel flags to grab a list of genes for further review
ccf_df[(ccf_df['structure abbreviation']=='AD') & (ccf_df['is_in_brain1_gene_panel'] | ccf_df['is_in_brain3_gene_panel'])]

## Add flag column for Nagalski *et al.* 2016 marker genes

In [15]:
nagalski_csv = pd.read_csv(os.path.join(root_dir,'resources/Nagalski_2016_Supplementary_Data_2_mouse_nuclei_specific_gene_table.csv'))

In [16]:
# want to be able to index into rows via gene name
nagalski_csv.set_index('Gene symbol', inplace=True)

In [17]:
nagalski_csv

### Display abbrv:fullname pairs for CCFv3 structures

In [18]:
# display abbv + full names for each CCFv3 structure
ccf_names = ccf_df[['structure abbreviation','full structure name']]
df_no_duplicates = ccf_names.drop_duplicates(subset=['structure abbreviation'])
df_sorted = df_no_duplicates.sort_values(by='structure abbreviation')
df_sorted

### Manual mapping dictionaries of CCFv3 <--> Nagalski structure abbrv

In [19]:
# display abbrv lists to manually compare
print(sorted(nagalski_csv.columns))
print(sorted(ccf_df['structure abbreviation'].unique()))

In [20]:
# manual mapping from Nagalski to CCFv3 thalamic nuclei abbreviations (by MT, 2023-10-26)
nagalski_to_ccfv3_structure_abbrv_mapping = {
    'AD': 'AD',
    'AM':['AMd','AMv'],
    'AV':'AV',
    'CL':'CL',
    'CM':'CM',
    'DLG':['LGd-co', 'LGd-ip', 'LGd-sh'],
    'IAD':'IAD',
    'IGL':'IGL',
    'IMD':'IMD',
    'LD':'LD',
    'LHb':'LH',
    'LP':'LP',
    'MD':'MD',
    'MGN':['MGd', 'MGm', 'MGv'],
    'MHb':'MH',
    'PC':'PCN',
    'PF':'PF',
    'PG':'LGv', # from Nagalski2016 pg 2497: "PG (former ventral lateral geniculate nucleus)"
    'PIL':'PIL',
    'PP':'PP',
    'PT':'PT',
    'PVA':'PVT',
    'PVP':'PVT',
    'Po':'PO',
    'RT':'RT',
    'Re':'RE',
    'ReA':'RE',
    'Rh':'RH',
    'SG':'SGN',
    'SPF':['SPA', 'SPFm', 'SPFp'],
    'Sub':'SMT', # from Nagalski2016 pg 2494: "Sub: Submedius thalamic nucleus"
    'VA/VL':'VAL',
    'VM':'VM',
    'VPL/VPM':['VPL', 'VPM'],
    'VPPC':['VPLpc', 'VPMpc'],
    'ZI':'ZI'
}

In [21]:
# manual mapping from CCFv3 to Nagalski thalamic nuclei abbreviations (by MT, 2023-10-26)
ccfv3_to_nagalski_structure_abbrv_mapping = {
    'AD': 'AD',
    'AMd':'AM',
    'AMv':'AM',
    'AV':'AV',
    'CL':'CL',
    'CM':'CM',
    'Eth':None,
    'IAD':'IAD',
    'IAM':None,
    'IGL':'IGL',
    'IMD':'IMD',
    'IntG':None,
    'LD':'LD',
    'LGd-co':'DLG',
    'LGd-ip':'DLG',
    'LGd-sh':'DLG',
    'LGv':'PG', # from Nagalski2016 pg 2497: "PG (former ventral lateral geniculate nucleus)"
    'LH':'LHb',
    'LP':'LP',
    'MD':'MD',
    'MGd':'MGN',
    'MGm':'MGN',
    'MGv':'MGN',
    'MH':'MHb',
    'PCN':'PC',
    'PF':'PF',
    'PIL':'PIL',
    'PO':'Po',
    'POL':None,
    'PP':'PP',
    'PR':None,
    'PT':'PT',
    'PVT':['PVA','PVP'],
    'PoT':None,
    'RE':['Re','ReA'],
    'RH':'Rh',
    'RT':'RT',
    'SGN':'SG',
    'SMT':'Sub', # from Nagalski2016 pg 2494: "Sub: Submedius thalamic nucleus"
    'SPA':'SPF',
    'SPFm':'SPF',
    'SPFp':'SPF',
    'SubG':None,
    'VAL':'VA/VL',
    'VM':'VM',
    'VPL':'VPL/VPM',
    'VPLpc':'VPPC',
    'VPM':'VPL/VPM',
    'VPMpc':'VPPC',
    'Xi':None,
    'ZI':'ZI'  
}

### Fill in column to flag Nagalski marker genes

In [22]:
ccf_df_with_nagalski = ccf_df.copy()

In [23]:
# Check if any of the Wang et al 2020 marker genes were also flagged by 
# Nagalski et al 2016 as expressing in that thalamic nucleus
# 
# I don't *think* there's a good way to vectorize this, given that I need to do
# checks on the structure abbreviations before I can use them to index in ...
for ind, row in ccf_df_with_nagalski.iterrows():
    ccf_abbrv = row['structure abbreviation']
    gene = row['gene symbol']
    
    if gene not in nagalski_csv.index:
        expr_flag = False
    else:
        nagalski_abbrv = ccfv3_to_nagalski_structure_abbrv_mapping[ccf_abbrv]
        
        if nagalski_abbrv is None:
            expr_flag = False
        else:
            expr_flag = nagalski_csv.loc[gene, nagalski_abbrv].any() # account for multiple mapped abbrv
    
    ccf_df_with_nagalski.loc[ind,'is_expressed_Nagalski2016'] = expr_flag

In [24]:
ccf_df_with_nagalski

## Save out as csv resource file

In [25]:
ccf_df_with_nagalski.to_csv(os.path.join(root_dir,'resources/thalamic_nuclei_marker_genes_from_Wang2020.csv'), index=False)